<a href="https://colab.research.google.com/github/Olayinka321/Big_Data_Project/blob/main/Agboola_Olayinka_BigDataCW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip uninstall pyspark -y
!pip install pyspark==3.0.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 25.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044161 sha256=de36a87857c8e6ef6f9cf735bc533cf97d4522de20f5a1b1a020ebead3561393
  Stored in directory: /root/.cache/pip/wheels/b1/bb/8b/ca24d3f756f2ed967225b0871898869db676eb5846df5adc56
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
!pip install findspark


In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("global_terrorism").getOrCreate()


In [ ]:
csv_path ='/content/sample_data/global_terrorism.csv'

df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.head()
df.show(10)

+----------+-----+------+----+----------+--------+----------+-------+------------------+------+--------------------+----------+-------------+----------+-----------+-----------+--------+---------------+--------------------+-----+-----+-----+---------+-----------+----------------+--------+-------+-------+-----------+--------------------+-----------+---------------+-----------+---------------+---------+--------------------+------------+--------------------+--------------------+--------------------+-------+------------------+---------+-------------+------------+----------------+-----+-------+-------+-----------+---------+-------------+------------+----------------+-----+-------+-------+-----------+--------------------+--------+------+---------+------+---------+--------------------+-----------+-----------+-----------+----------+------+--------+-------+---------+-------------+------+----------+--------------+------+----------+--------------+---------+---------+-------------+------------+----

In [ ]:
#to check number of rows
df.count()

181691

In [ ]:
#To check number of columns
len(df.columns)

135

In [ ]:
#Data Preprocessing

import pyspark.sql.functions as F
from pyspark.mllib.stat import Statistics

# Drop the columns
df = df.drop('approxdate','resolution','attacktype3','target2','corp2')
df = df.drop('ransomamt','ransompaid','ransompaidus','ransomamtus')
df = df.drop('ransomnote','nreleased','hostkidoutcome','hostkidoutcome_txt')
df = df.drop('attacktype3_txt','targtype','targsubtype','natlty2','natlty3')
df = df.drop('gname2','gsubname3','claimmode','claimmode2_txt','nhours')
df = df.drop('natlty3_txt','gsubname','claimmode_txt','weaptype2','ndays')
df = df.drop('nhostkidus','weaptype2_txt','claim2','gsubname2','gname3')
df = df.drop('weapsubtype2_txt','weapsubtype2','targtype2_txt','targsubtype2_txt')
df = df.drop('weapsubtype3_txt','weapsubtype3','weaptype3','weaptype3_txt')
df = df.drop('weapsubtype4_txt','weapsubtype4','weaptype4','weaptype4_txt')
df = df.drop('targtype2','targsubtype2','location','natlty2_txt','targtype3','targtype3_txt')
df = df.drop('claimed','claimmode2','claim3','claimmode3')
df = df.drop('claimmode3_txt','compclaim','attacktype2','attacktype2_txt')
df = df.drop('alternative','alternative_txt','guncertain1','guncertain2')
df = df.drop('guncertain3','propextent','propextent_txt','propvalue')
df = df.drop('propcomment','ishostkid','extended','nhostkid','divert','kidhijcountry')
df = df.drop('targsubtype3','targsubtype3_txt','corp3','target3')



# Check for empty cells
#print(df.filter(F.col("nkill").isNull()).count())

# Ignore empty cells
#df = df.filter(F.col("nkill").isNotNull())
#means = Statistics.colMeans(df.toDF())
#df = df.fillna(means)
#means = df.stat.get_column_means()
#df = df.fillna(means)
# Impute empty cells with the column means
#df = df.fillna(df.stat.colMeans())

# Drop rows with empty cells
#df = df.na.drop()


# Show the first 10 rows of the DataFrame after dropping missing data
df.show(10)

+----------+-----+------+----+-------+------------------+------+--------------------+----------+-------------+----------+-----------+-----------+--------+--------------------+-----+-----+-----+---------+--------+-------+-------+-----------+--------------------+---------+--------------------+------------+--------------------+--------------------+--------------------+-------+------------------+--------------------+--------------------+----------+------+--------+---------+-------------+------------+--------------------+--------------------+-----+-------+--------+------+--------+--------+--------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+
|   eventid|iyear|imonth|iday|country|       country_txt|region|          region_txt| provstate|         city|  latitude|  longitude|specificity|vicinity|             summary|crit1|crit2

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
# List of categorical columns
categorical_columns = ["country_txt", "attacktype1_txt", "targtype1_txt", "weaptype1_txt"]

# List of numerical columns
numerical_columns = ['iyear', 'imonth']

# Converting categorical columns to index using StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in categorical_columns]

# Using VectorAssembler to combine the categorical columns and numerical columns into a single vector column
assembler = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numerical_columns, outputCol="features")

# Define the pipeline
pipeline = Pipeline(stages=indexers + [assembler])

#Missing value drop
df = df.na.drop()


# Convert the success column to a boolean type
#df = df.withColumn("success", df["success"].cast("boolean"))

# Fit the pipeline to training data
df_transformed = pipeline.fit(df).transform(df)

# Show the first 10 rows of the transformed DataFrame
df_transformed.show(10)


+----------+-----+------+----+-------+-------------+------+-------------+-----------+-------------+---------+-----------+-----------+--------+--------------------+-----+-----+-----+---------+--------+-------+-------+-----------+--------------------+---------+--------------------+------------+--------------------+--------------------+--------------------+-------+-------------+--------------------+--------------------+----------+------+--------+---------+-------------+------------+--------------------+--------------------+-----+-------+--------+------+--------+--------+--------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+---------------------+-------------------+-------------------+--------------------+
|   eventid|iyear|imonth|iday|country|  country_txt|region|   region_txt|  provstate|         city

In [ ]:
from pyspark.sql.functions import col

#df = df.withColumn("success", col("success").cast("integer"))


# Split the data into training and test sets
(train_data, test_data) = df_transformed.randomSplit([0.7, 0.3])

# Define the Logistic Regression model
lr = LogisticRegression(labelCol="success", featuresCol="features")

# Train the model
lr_model = lr.fit(train_data)

# Make predictions
predictions = lr_model.transform(test_data)

# Select (prediction, true label) and compute test error
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="success", metricName="areaUnderROC")

# Evaluate the model and print the AUC
auc = evaluator.evaluate(predictions)
print(f"AUC of Logistic Regression model is: {auc}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

# Show the first 10 rows of the predictions DataFrame
predictions.show(10)


AUC of Logistic Regression model is: 0.6852275616577675
Coefficients: [0.11941818458094872,0.20340864374129194,0.00945288455243359,0.4150702121120542,-0.00035066149593230265,-0.0007978015054886149]
Intercept: 1.9087034973589292
+----------+-----+------+----+-------+-------------+------+-------------+----------+-------------+---------+-----------+-----------+--------+--------------------+-----+-----+-----+---------+--------+-------+-------+-----------+--------------------+---------+--------------------+------------+--------------------+--------------------+--------------------+-------+-------------+--------------------+--------------------+----------+------+--------+---------+-------------+------------+--------------------+--------------------+-----+-------+--------+------+--------+--------+--------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+------------------

In [ ]:
# Define the Random Forest model
rf = RandomForestClassifier(labelCol="success", featuresCol="features",maxBins=200)

# Train the model
rf_model = rf.fit(train_data)

# Make predictions
predictions_rf = rf_model.transform(test_data)

# Select (prediction, true label) and compute test error
evaluator_rf = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="success", metricName="areaUnderROC")

# Evaluate the model and print the AUC
auc_rf = evaluator_rf.evaluate(predictions_rf)
print(f"AUC of Random Forest model is: {auc_rf}")

# Show the first 10 rows of the predictions DataFrame
predictions_rf.show(10)


AUC of Random Forest model is: 0.7813374014747012
+----------+-----+------+----+-------+-------------+------+-------------+----------+-------------+---------+-----------+-----------+--------+--------------------+-----+-----+-----+---------+--------+-------+-------+-----------+--------------------+---------+--------------------+------------+--------------------+--------------------+--------------------+-------+-------------+--------------------+--------------------+----------+------+--------+---------+-------------+------------+--------------------+--------------------+-----+-------+--------+------+--------+--------+--------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-----------------+---------------------+-------------------+-------------------+--------------------+--------------------+--------------------+----------+

In [ ]:
# Define the Support Vector Machine model
svm = LinearSVC(labelCol="success", featuresCol="features")

# Train the model
svm_model = svm.fit(train_data)

# Make predictions
predictions_svm = svm_model.transform(test_data)

# Select (prediction, true label) and compute test error
evaluator_svm = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="success", metricName="areaUnderROC")

# Evaluate the model and print the AUC
auc_svm = evaluator_svm.evaluate(predictions_svm)
print(f"AUC of Support Vector Machine model is: {auc_svm}")

# Show the first 10 rows of the predictions DataFrame
predictions_svm.show(10)


AUC of Support Vector Machine model is: 0.7017543859649124
+----------+-----+------+----+-------+-------------+------+-------------+----------+-------------+---------+-----------+-----------+--------+--------------------+-----+-----+-----+---------+--------+-------+-------+-----------+--------------------+---------+--------------------+------------+--------------------+--------------------+--------------------+-------+-------------+--------------------+--------------------+----------+------+--------+---------+-------------+------------+--------------------+--------------------+-----+-------+--------+------+--------+--------+--------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-----------------+---------------------+-------------------+-------------------+--------------------+--------------------+----------+
|   eventid

In [ ]:
#to help the efficiency of the models by selecting only the columns

In [ ]:
# Select the columns
selected_columns = ["iyear", "imonth", "country_txt", "region_txt", "city", "attacktype1_txt",
                    "targtype1_txt", "targsubtype1_txt", "target1", "natlty1_txt", "gname", "motive", "success"]

df = df.select(selected_columns)
df.show()

+-----+------+-------------+-------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------+
|iyear|imonth|  country_txt|   region_txt|          city|     attacktype1_txt|       targtype1_txt|    targsubtype1_txt|             target1|  natlty1_txt|               gname|              motive|success|
+-----+------+-------------+-------------+--------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------+
| 1970|     1|United States|North America|         Cairo|       Armed Assault|              Police|Police Building (...|Cairo Police Head...|United States|  Black Nationalists|To protest the Ca...|      1|
| 1970|     1|United States|North America|       Madison|Facility/Infrastr...|            Military|Military Recruiti...|R.O.T.C. offices ...|United States|     New Year's Gang|

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Columns to be indexed and encoded
categorical_columns = ["country_txt", "region_txt", "city", "attacktype1_txt", "targtype1_txt",
                       "targsubtype1_txt", "target1", "natlty1_txt", "gname", "motive"]

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="keep") for column in categorical_columns]
encoders = [OneHotEncoder(inputCol=column+"_index", outputCol=column+"_ohe") for column in categorical_columns]

pipeline = Pipeline(stages=indexers + encoders)

df_encoded = pipeline.fit(df).transform(df)


In [ ]:
#To create a vector containing all the feature columns to feed into the model

In [ ]:
from pyspark.ml.feature import VectorAssembler

input_cols = ["iyear", "imonth"] + [column+"_ohe" for column in categorical_columns]
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

df_encoded = assembler.transform(df_encoded)


In [ ]:
# Splitting of the dataset into training and testing set
train_data, test_data = df_encoded.randomSplit([0.7, 0.3], seed=42)


In [ ]:
#To define the Logistic Regression model
lr = LogisticRegression(labelCol="success", featuresCol="features")

# Training the LR model
lr_model = lr.fit(train_data)

#To make predictions
predictions_lr = lr_model.transform(test_data)

# Selection of (prediction, true label) and compute test error
evaluator_lr = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="success", metricName="areaUnderROC")

# Evaluation of the model and to print the AUC
auc_lr = evaluator_lr.evaluate(predictions_lr)
print(f"AUC of Logistic Regression model is: {auc_lr}")

# Show the first 10 rows of the predictions DataFrame
predictions_lr.show(10)


AUC of Logistic Regression model is: 0.816400580551524
+-----+------+-------------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------+-----------------+----------------+----------+---------------------+-------------------+----------------------+-------------+-----------------+-----------+------------+---------------+--------------+----------------+-------------------+-----------------+--------------------+-----------------+---------------+----------------+-----------------+--------------------+--------------------+--------------------+----------+
|iyear|imonth|  country_txt|   region_txt|       city|     attacktype1_txt|       targtype1_txt|    targsubtype1_txt|             target1|  natlty1_txt|               gname|              motive|success|country_txt_index|region_txt_index|city_index|attacktype1_txt_index|targtype1_txt_index|targsubtype1_txt_index|target1_ind

In [ ]:
from pyspark.ml.classification import LinearSVC

#To define the Support Vector Machine model
svm = LinearSVC(labelCol="success", featuresCol="features")

#Training of the model
svm_model = svm.fit(train_data)

#To make predictions
predictions_svm = svm_model.transform(test_data)

#Selection of (prediction, true label) and compute test error
evaluator_svm = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="success", metricName="areaUnderROC")

#Evaluationof the model and printing the AUC
auc_svm = evaluator_svm.evaluate(predictions_svm)
print(f"AUC of SVM model is: {auc_svm}")

#Showing the first 10 rows of the predictions DataFrame
predictions_svm.show(10)


AUC of SVM model is: 0.7070633768746977
+-----+------+-------------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------+-----------------+----------------+----------+---------------------+-------------------+----------------------+-------------+-----------------+-----------+------------+---------------+--------------+----------------+-------------------+-----------------+--------------------+-----------------+---------------+----------------+-----------------+--------------------+--------------------+----------+
|iyear|imonth|  country_txt|   region_txt|       city|     attacktype1_txt|       targtype1_txt|    targsubtype1_txt|             target1|  natlty1_txt|               gname|              motive|success|country_txt_index|region_txt_index|city_index|attacktype1_txt_index|targtype1_txt_index|targsubtype1_txt_index|target1_index|natlty1_txt_index|gname_index|mot

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# Define the Random Forest model
rf = RandomForestClassifier(labelCol="success", featuresCol="features", numTrees=100)

# Train the model
rf_model = rf.fit(train_data)

# Make predictions
predictions_rf = rf_model.transform(test_data)

# Select (prediction, true label) and compute test error
evaluator_rf = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="success", metricName="areaUnderROC")

# Evaluate the model and print the AUC
auc_rf = evaluator_rf.evaluate(predictions_rf)
print(f"AUC of Random Forest model is: {auc_rf}")

# Show the first 10 rows of the predictions DataFrame
predictions_rf.show(10)


AUC of Random Forest model is: 0.813014029995162
+-----+------+-------------+-------------+-----------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-------+-----------------+----------------+----------+---------------------+-------------------+----------------------+-------------+-----------------+-----------+------------+---------------+--------------+----------------+-------------------+-----------------+--------------------+-----------------+---------------+----------------+-----------------+--------------------+--------------------+--------------------+----------+
|iyear|imonth|  country_txt|   region_txt|       city|     attacktype1_txt|       targtype1_txt|    targsubtype1_txt|             target1|  natlty1_txt|               gname|              motive|success|country_txt_index|region_txt_index|city_index|attacktype1_txt_index|targtype1_txt_index|targsubtype1_txt_index|target1_index|nat

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator

feature_columns = ["country_txt_ohe", "region_txt_ohe", "city_ohe", "attacktype1_txt_ohe",
                   "targtype1_txt_ohe", "targsubtype1_txt_ohe", "target1_ohe",
                   "natlty1_txt_ohe", "gname_ohe", "motive_ohe"]

#Using the VectorAssembler to combine the feature columns into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features_vector")
train_data_assembled = assembler.transform(train_data)
test_data_assembled = assembler.transform(test_data)

#Setting up the Naive Bayes model
nb = NaiveBayes(featuresCol="features_vector", labelCol="success", predictionCol="prediction")

#Training the Naive Bayes model
nb_model = nb.fit(train_data_assembled)

#To Predict on the test set
predictions = nb_model.transform(test_data_assembled)

#Performance Evaluation of ROC AUC
evaluator = BinaryClassificationEvaluator(labelCol="success", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predictions)

#To show the output

print(f"ROC AUC: {roc_auc}")


ROC AUC: 0.32873730043541366


In [ ]:
from pyspark.ml.feature import PCA, VectorAssembler

# List of encoded features
feature_columns = [column+"_ohe" for column in categorical_columns]

# Assembling encoded features into a single vector column
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features_vector")
df_assembled = assembler.transform(df_encoded)

# Applying PCA
num_principal_components = 13
pca = PCA(k=num_principal_components, inputCol="features_vector", outputCol="pca_features")
pca_model = pca.fit(df_assembled)
df_pca = pca_model.transform(df_assembled)

# Show transformed data
df_pca.select("pca_features").show()


+--------------------+
|        pca_features|
+--------------------+
|[-1.2112337161875...|
|[-1.5188752712147...|
|[-1.5926528820907...|
|[-1.6359463675041...|
|[-1.4494800091229...|
|[-1.5364736781838...|
|[-1.2744952660037...|
|[-1.3572971155669...|
|[-1.6713152057417...|
|[-1.2093709792562...|
|[-1.3086171397441...|
|[-1.3526140105896...|
|[-1.5929733674757...|
|[-1.1862500690598...|
|[-1.4556383886623...|
|[-1.4556383886623...|
|[-1.2601146225341...|
|[-1.5373263739132...|
|[-1.6057361622243...|
|[-1.2872894370008...|
+--------------------+
only showing top 20 rows

